In [9]:
import torch
from torch import tensor
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from ast import literal_eval
from py_experimenter.experimenter import PyExperimenter
import seaborn as sns
import torch
import mysql.connector
import openml


from py_experimenter.database_connector_mysql import DatabaseConnectorMYSQL

def connect(self):

    db = mysql.connector.connect(
        host="db01-kiml.kiml.ifi.lmu.de",
        user="jonas_h",
        password="thiswillnotactuallybevalidforverylongsodontcountonit",
        database="jonas_test",
        ssl_disabled=False,
    )
    return db


def _start_transaction(self, connection, readonly=False):
    if not readonly:
        connection.start_transaction()


DatabaseConnectorMYSQL.connect = connect
DatabaseConnectorMYSQL._start_transaction = _start_transaction

%matplotlib inline

In [10]:
experimenter = PyExperimenter(
    experiment_configuration_file_path="./experiments/config/config.yml",
)
exp_frame = experimenter.get_table()
exp_frame = exp_frame[exp_frame.fraction_cal_samples >= 0.19999]
exp_frame = exp_frame[exp_frame.openml_id != 39]


2024-11-18 13:00:41,321  | py-experimenter - INFO     | Found 5 keyfields
2024-11-18 13:00:41,322  | py-experimenter - INFO     | Found 10 resultfields
2024-11-18 13:00:41,323  | py-experimenter - WARNING  | No logtables given
2024-11-18 13:00:41,323  | py-experimenter - WARNING  | No custom section defined in config
2024-11-18 13:00:41,323  | py-experimenter - WARNING  | No codecarbon section defined in config
2024-11-18 13:00:41,332  | py-experimenter - INFO     | Initialized and connected to database


In [11]:
import numpy as np
datasets = {}
num_classes = {}
dataset_renamer = {}
for id in exp_frame.openml_id.unique():
    dataset = openml.datasets.get_dataset(id.item())
    target_attribute = dataset.default_target_attribute
    X, y, _, _ = dataset.get_data(target=target_attribute)
    datasets[id] = dataset
    dataset_renamer[id] = dataset.name
    num_classes[id] = len(np.unique(y))

/home/jonas/Documents/Research/dyad_ranking/torch_plnet/.venv/lib/python3.11/site-packages/openml/utils.py:461: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return func(*args, **kwargs)
/home/jonas/Documents/Research/dyad_ranking/torch_plnet/.venv/lib/python3.11/site-packages/openml/utils.py:461: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  return func(*args, 

In [12]:
exp_frame[exp_frame.model=="plnet_cross_instance"].status.value_counts()

status
done    150
Name: count, dtype: int64

In [13]:
df = exp_frame

exp_frame["dataset"] =  exp_frame["openml_id"].replace(dataset_renamer)
exp_frame["num_classes"] = exp_frame["openml_id"].replace(num_classes)

exp_frame["dataset"] = exp_frame["dataset"] + " (" + exp_frame["num_classes"].astype(str) + ")" 

# group_cols = ['dataset', 'alpha', 'fraction_cal_samples', 'model']
group_cols = ['dataset', 'alpha', 'model']
metrics_max = ['score_acc', 'coverage_mean']
metrics_min = ['efficiency_mean']
metrics = metrics_max + metrics_min

grouped_df = df.groupby(group_cols).agg({metric: 'mean' for metric in metrics}).reset_index()
agg_dict = {metric: 'max' for metric in metrics_max}
agg_dict.update({metric: 'min' for metric in metrics_min})
best_values = grouped_df.groupby(['dataset', 'alpha']).agg(agg_dict).reset_index()

In [14]:
def generate_latex_table_with_lines(df, group_cols, value_cols, best_values):
    """Generate a LaTeX table with multirow entries, bold the best values, and add lines between consecutive multirows."""
    
    def format_value(value, group, metric, max=True):
        """Format the value, making the highest value bold."""
        max_value = best_values.loc[
            (best_values['dataset'] == group['dataset']) & 
            (best_values['alpha'] == group['alpha']), metric].values[0]
            # (best_values['fraction_cal_samples'] == group['fraction_cal_samples']), metric].values[0]
        if value == max_value:
            return f"\\textbf{{{value:.4f}}}"
        else:
            return f"{value:.4f}"
    
    def recursive_build(df, group_cols):
        # if len(group_cols) == 1:
            # Base case: only one group left, print it directly
            latex_str = ""
            prev_row = None
            for row_id, row in df.iterrows():
                latex_str_tmp = ""
                col_id_list = []
                for col_id, col in enumerate(group_cols):
                    if prev_row is None or row[col] != prev_row[col]:
                        col_id_list.append(col_id)
                        filter_cols = group_cols[:col_id+1]
                        indices = []
                        values = []
                        for fcol in filter_cols:
                            indices.append(fcol)
                            values.append(row[fcol])
                        criteria = dict(zip(indices, values))
                        # Filter the DataFrame using query
                        mask = pd.Series([True] * len(df))

                        # Iterate over criteria to apply conditions
                        for key, value in criteria.items():
                            mask = mask & (df[key] == value)

                        # Filter the DataFrame
                        filtered_df = df[mask]

                        length = len(filtered_df)


                        latex_str_tmp += f" \\multirow{{{length}}}{{*}}{{{row[col]}}} & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"

                        # if prev_row is not None and col_id < len(group_cols) - 1:
                        # latex_str_tmp += " \\\\ \cline{%d-%d} \\\\" % (col_id + 1, len(group_cols) + len(value_cols))

                    else:
                        latex_str_tmp += " & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"
                if row_id > 0 and group_cols[min(col_id_list)] != "model":
                    latex_str += "\\cline{%d-%d}" % (min(col_id_list)+1, len(group_cols) + len(value_cols))
                latex_str += latex_str_tmp
                indices = []
                values = []
                prev_row = row
            return latex_str
        
    # Start recursive building
    latex_body = recursive_build(df, group_cols)

    # Complete LaTeX table
    num_columns = len(group_cols) + len(value_cols)
    col_format = 'l' * len(group_cols) + 'r' * len(value_cols)
    latex_table = f"""
\\begin{{tabular}}{{{col_format}}}
\\toprule
{' & '.join(group_cols)} & {' & '.join(value_cols)} \\\\
\\midrule
{latex_body}
\\bottomrule
\\end{{tabular}}
"""
    return latex_table

# Generate the LaTeX table with bold formatting and lines
latex_table = generate_latex_table_with_lines(grouped_df, group_cols, metrics, best_values)

# Print or save the LaTeX table
print(latex_table.replace("_", "\_"))



\begin{tabular}{lllrrr}
\toprule
dataset & alpha & model & score\_acc & coverage\_mean & efficiency\_mean \\
\midrule
 \multirow{12}{*}{dermatology (6)} &  \multirow{4}{*}{0.05} &  \multirow{1}{*}{classifier\_nn} & 0.9703 & 0.9527 & \textbf{0.9703} \\
 &  &  \multirow{1}{*}{plnet} & 0.9595 & 0.9527 & 3.6905 \\
 &  &  \multirow{1}{*}{plnet\_cross\_instance} & 0.9622 & 0.9581 & 1.0392 \\
 &  &  \multirow{1}{*}{random\_forest} & \textbf{0.9757} & \textbf{0.9622} & 0.9797 \\
\cline{2-6} &  \multirow{4}{*}{0.1} &  \multirow{1}{*}{classifier\_nn} & 0.9703 & 0.9189 & 0.9243 \\
 &  &  \multirow{1}{*}{plnet} & 0.9595 & 0.9203 & 3.1865 \\
 &  &  \multirow{1}{*}{plnet\_cross\_instance} & 0.9622 & \textbf{0.9311} & 0.9689 \\
 &  &  \multirow{1}{*}{random\_forest} & \textbf{0.9757} & 0.9149 & \textbf{0.9189} \\
\cline{2-6} &  \multirow{4}{*}{0.2} &  \multirow{1}{*}{classifier\_nn} & 0.9703 & \textbf{0.8392} & 0.8405 \\
 &  &  \multirow{1}{*}{plnet} & 0.9595 & 0.8216 & 2.8027 \\
 &  &  \multirow{1}

In [15]:
grouped_df

,dataset,alpha,model,score_acc,coverage_mean,efficiency_mean
0,dermatology (6),0.05,classifier_nn,0.970270,0.952703,0.970269
1,dermatology (6),0.05,plnet,0.959459,0.952703,3.690539
2,dermatology (6),0.05,plnet_cross_instance,0.962162,0.958108,1.039190
3,dermatology (6),0.05,random_forest,0.975675,0.962162,0.979729
4,dermatology (6),0.10,classifier_nn,0.970270,0.918919,0.924324
5,dermatology (6),0.10,plnet,0.959459,0.920270,3.186489
6,dermatology (6),0.10,plnet_cross_instance,0.962162,0.931081,0.968919
7,dermatology (6),0.10,random_forest,0.975675,0.914865,0.918919
8,dermatology (6),0.20,classifier_nn,0.970270,0.839189,0.840541
9,dermatology (6),0.20,plnet,0.959459,0.821622,2.802703


In [16]:
grouped_df

,dataset,alpha,model,score_acc,coverage_mean,efficiency_mean
0,dermatology (6),0.05,classifier_nn,0.970270,0.952703,0.970269
1,dermatology (6),0.05,plnet,0.959459,0.952703,3.690539
2,dermatology (6),0.05,plnet_cross_instance,0.962162,0.958108,1.039190
3,dermatology (6),0.05,random_forest,0.975675,0.962162,0.979729
4,dermatology (6),0.10,classifier_nn,0.970270,0.918919,0.924324
5,dermatology (6),0.10,plnet,0.959459,0.920270,3.186489
6,dermatology (6),0.10,plnet_cross_instance,0.962162,0.931081,0.968919
7,dermatology (6),0.10,random_forest,0.975675,0.914865,0.918919
8,dermatology (6),0.20,classifier_nn,0.970270,0.839189,0.840541
9,dermatology (6),0.20,plnet,0.959459,0.821622,2.802703
